In [3]:
# !pip install gensim

In [4]:
from gensim import models
from gensim.models import Word2Vec
import gzip
from dataclasses import dataclass
from typing import Iterator
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords


In [5]:
@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("../Data/news.txt.gz"))

In [6]:
sentenses = []
splitted_sentenses = []
stop_words = set(stopwords.words('russian'))

for text in texts:
    words = nltk.word_tokenize(text=text.text, language="russian")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    sentenses.append(' '.join(filtered_words[j] for j in range (0, len(filtered_words))))
    splitted_sentenses.append(filtered_words)


In [7]:
sentenses[0]

'Парусная гонка Giraglia Rolex Cup пройдет Средиземном море 64-й раз. Победители соревнования , проводимого 1953 года Yacht Club Italiano , помимо других призов традиционно получают подарок часы швейцарского бренда Rolex . сообщается пресс-релизе , поступившем редакцию « Ленты.ру » среду , 8 мая . Rolex Yacht-Master 40 Фото : пресс-служба Mercury Соревнования будут проходить 10 18 июня . Первый этап : ночной переход Сан-Ремо Сен-Тропе 10-11 июня ( дистанция 50 морских миль — около 90 километров ) . Второй этап : серия прибрежных гонок бухте Сен-Тропе 11 14 июня . Финальный этап пройдет 15 18 июня : оффшорная гонка маршруту Сен-Тропе — Генуя ( 243 морских мили — 450 километров ) . Маршрут проходит скалистый остров Джиралья северу Корсики завершается Генуе.Регата , 1997 года проходящая поддержке Rolex , считается одной самых значительных яхтенных гонок Средиземноморье . году ожидается участие трех российских экипажей .'

In [8]:
model = Word2Vec(sentences=splitted_sentenses, workers=8)

In [9]:
model.save("word2vec.model")

In [10]:
def average_word_vectors(sentence, model):
    word_vectors = []
    
    for word in sentence:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [11]:
vectors = [average_word_vectors(sentence, model) for sentence in splitted_sentenses]

In [12]:
print(vectors[0])

[-0.24972625  0.15228556 -0.0655603   0.3427927   0.43298772 -0.6190261
  0.11213136  0.49906248 -0.71044374 -0.7572064   0.21235925 -0.21054778
 -0.3506796   0.1666163   0.21358886  0.09529348  0.4474777  -0.03525523
 -0.24684346 -0.9685215   0.22220147  0.55748487  0.7496978  -0.26424608
  0.17677766 -0.19013888 -0.26553094 -0.00117834 -0.523602    0.21474867
  0.33892655  0.08588235  0.23401904 -0.45223358  0.02865029  0.5828208
 -0.06534346 -0.39434323  0.11420307 -0.09098113  0.29987088 -0.21736923
  0.0498212  -0.30486494  0.3850064   0.16141118 -0.5291881  -0.05188745
  0.40990278 -0.20734395  0.01585291 -0.24456283 -0.31975678  0.16815147
 -0.18735014  0.30478457 -0.01798353 -0.16565894 -0.19530725 -0.02502478
 -0.16379511 -0.0043614   0.01051776  0.0394912  -0.2778595   0.6200434
 -0.16141307  0.17411736 -0.60173     0.15019858 -0.3187736   0.10178133
  0.36256802 -0.01879267  0.2954343  -0.47412914  0.5573273  -0.55865055
 -0.04683385 -0.2585479  -0.0087524  -0.05906957 -0.28

In [13]:
labels = []
for text in texts:
    labels.append(text.label)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size=0.2, random_state=77)

In [15]:
SVC_model = SVC()  
SVC_model.fit(np.asarray(X_train), np.asarray(y_train))

SVC()

In [16]:
y_pred = SVC_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.691
              precision    recall  f1-score   support

    business       0.00      0.00      0.00        64
     culture       0.67      0.73      0.70       273
   economics       0.63      0.83      0.72       265
      forces       0.58      0.69      0.63       170
        life       0.67      0.73      0.70       262
       media       0.64      0.57      0.60       295
     science       0.67      0.70      0.68       282
       sport       0.94      0.89      0.91       318
       style       0.60      0.16      0.26        37
      travel       0.50      0.03      0.06        34

    accuracy                           0.69      2000
   macro avg       0.59      0.53      0.53      2000
weighted avg       0.67      0.69      0.67      2000



In [17]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(sentenses)
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=77)

In [18]:
SVC_model = SVC()  
SVC_model.fit(X_train_tfidf, np.asarray(y_train))

SVC()

In [19]:
y_pred = SVC_model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8365
              precision    recall  f1-score   support

    business       1.00      0.08      0.14        64
     culture       0.91      0.89      0.90       273
   economics       0.79      0.92      0.85       265
      forces       0.84      0.72      0.78       170
        life       0.70      0.93      0.80       262
       media       0.84      0.82      0.83       295
     science       0.81      0.89      0.85       282
       sport       0.98      0.94      0.96       318
       style       1.00      0.30      0.46        37
      travel       1.00      0.29      0.45        34

    accuracy                           0.84      2000
   macro avg       0.89      0.68      0.70      2000
weighted avg       0.85      0.84      0.82      2000

